In [4]:
import pandas as pd
import numpy as np
import math
import matplotlib
import csv
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from matplotlib import pyplot
import pandas_profiling as pf
%matplotlib inline

In [5]:
missing_feature_map = {'motor_vehicle_license_total': {'op': 'ADD',
  'src_features': ['motor_vehicle_license', 'motor_veh_oper_license']},
 'state_igr_gen_sup': {'op': 'ADD',
  'src_features': ['state_igr_tax_relief', 'state_igr_oth_gen_sup']},
 'chg_total_elem_education': {'op': 'ADD',
  'src_features': ['chg_elem_ed_sch_lunch',
   'chg_elem_ed_tuition',
   'chg_elem_ed_nec']},
 'chg_highways': {'op': 'ADD',
  'src_features': ['chg_regular_highways', 'chg_toll_highways']},
 'prop_sale_total': {'op': 'ADD',
  'src_features': ['prop_sale_hous_com_dev', 'prop_sale_other']},
 'total_other_capital_outlays': {'op': 'SUBSTRACT',
  'src_features': ['total_capital_outlays', 'total_construction']},
 'general_capital_outlay_other': {'op': 'SUBSTRACT',
  'src_features': ['general_capital_outlay', 'general_construction']},
 'air_trans_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['air_trans_direct_expend', 'air_trans_cap_outlay']},
 'misc_com_activ_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['misc_com_activ_tot_exp', 'misc_com_activ_cap_out']},
 'correct_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['correct_direct_exp', 'correct_cap_outlay']},
 'elem_educ_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['elem_educ_direct_exp', 'elem_educ_cap_outlay']},
 'total_educ_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['total_educ_direct_exp', 'total_educ_cap_outlay']},
 'higher_ed_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['higher_ed_direct_exp', 'higher_ed_cap_outlay']},
 'educ_nec_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['educ_nec_direct_expend', 'educ_nec_cap_outlay']},
 'emp_sec_adm_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['emp_sec_adm_direct_exp', 'emp_sec_adm_cap_outlay']},
 'fin_admin_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['fin_admin_direct_exp', 'fin_admin_cap_outlay']},
 'fire_prot_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['fire_prot_direct_exp', 'fire_prot_cap_outlay']},
 'judicial_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['judicial_direct_expend', 'judicial_cap_outlay']},
 'cen_staff_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['cen_staff_direct_exp', 'cen_staff_cap_outlay']},
 'gen_pub_bldg_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['gen_pub_bldg_total_exp', 'gen_pub_bldg_cap_out']},
 'health_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['health_direct_expend', 'health_capital_outlay']},
 'total_hospital_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['total_hospital_total_exp', 'total_hospital_cap_out']},
 'own_hospital_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['own_hospital_total_exp', 'own_hospital_cap_out']},
 'hosp_other_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['hosp_other_direct_exp', 'hosp_other_cap_outlay']},
 'total_highways_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['total_highways_dir_exp', 'total_highways_cap_out']},
 'regular_hwy_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['regular_hwy_direct_exp', 'regular_hwy_cap_outlay']},
 'toll_hwy_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['toll_hwy_total_expend', 'toll_hwy_cap_outlay']},
 'hous___com_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['hous___com_direct_exp', 'hous___com_cap_outlay']},
 'libraries_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['libraries_direct_exp', 'libraries_cap_outlay']},
 'natural_res_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['natural_res_direct_exp', 'natural_res_cap_outlay']},
 'parks___rec_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['parks___rec_direct_exp', 'parks___rec_cap_outlay']},
 'police_prot_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['police_prot_direct_exp', 'police_prot_cap_outlay']},
 'prot_insp_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['prot_insp_direct_exp', 'prot_insp_cap_outlay']},
 'public_welf_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['public_welf_direct_exp',
   'public_welf_cash_asst',
   'public_welf_cap_outlay']},
 'welf_ins_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['welf_ins_total_exp', 'welf_ins_cap_outlay']},
 'welf_nec_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['welf_nec_direct_expend', 'welf_nec_cap_outlay']},
 'sewerage_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['sewerage_direct_expend', 'sewerage_cap_outlay']},
 'sw_mgmt_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['sw_mgmt_direct_expend', 'sw_mgmt_capital_outlay']},
 'water_trans_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['water_trans_direct_exp', 'water_trans_cap_outlay']},
 'general_nec_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['general_nec_direct_exp', 'general_nec_cap_outlay']},
 'liquor_stores_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['liquor_stores_tot_exp', 'liquor_stores_cap_out']},
 'total_util_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['total_util_total_exp',
   'total_util_inter_exp',
   'total_util_cap_outlay']},
 'water_util_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['water_util_total_exp',
   'water_util_inter_exp',
   'water_util_cap_outlay']},
 'elec_util_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['elec_util_total_exp',
   'elec_util_inter_exp',
   'elec_util_cap_outlay']},
 'gas_util_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['gas_util_total_exp',
   'gas_util_inter_exp',
   'gas_util_cap_outlay']},
 'trans_util_current_exp': {'op': 'SUBSTRACT',
  'src_features': ['trans_util_total_exp',
   'trans_util_inter_exp',
   'trans_util_cap_outlay']}}

In [6]:
all_data="/Users/lohith/Desktop/NEU_Courses/ThirdSemester/CS6510-ASD/data/TheGovernmentFinanceDatabase_AllData/all.csv"

# Type_Code
STATE = 0
COUNTY = 1 
CITY = 2                                 
TOWNSHIP = 3 
SPECIAL_DIST = 4
SCHOOL_DIST = 5   
FED_GOVT = 6 
ADD="ADD"
SUBSTRACT = "SUBSTRACT"

additionlist = []
substraction_list = []

In [7]:
def save_to_csv(d,file_name):
    """
        Writes contents of the given dict into a csv file
        Ex: d= {'Name':138,'Population':5,...}
        
        Args:
            d (dict)      : dictionary containing key value pair where key is the 
                            feature name and value is the number of missing zeros or nans
            filename(str) : output file name (absolute path)
    """
    with open(file_name, 'w') as fp:
        fwriter = csv.writer(fp)
        for val in d.items():
            fwriter.writerow(val)

def count_negatives(df):
    """
        Calculate the number of negative values for the features in the dataframe
        This applies only to numeric dtypes
    
        Args:
            df (Pandas Dataframe)  : A pandas dataframe
        Returns:
            dict : dictionary containing key value pair where key is the 
                   feature name and value is the number of missing zeros 
    """
    return (df.loc[:, df.dtypes != object] < 0).sum(axis=0).to_dict() 

def count_zeros(df):
    """
        Calculate the number of zeros for the features in the dataframe
    
        Args:
            df (Pandas Dataframe)  : A pandas dataframe 
        Returns:
            dict : dictionary containing key value pair where key is the 
                   feature name and value is the number of missing zeros 
    """
    return (df == 0).sum(axis=0).to_dict() 

def count_nans(df):
    """
        Calculate the number of nans for the features in the dataframe
    
        Args:
            df (Pandas Dataframe)  : A pandas dataframe 
        Returns:
            dict : dictionary containing key value pair where key as feature
            and value as the nan count for that feature
    """
    return (len(df) - df.count()).to_dict()
    

def drop_features(df, threshold=0.5):
    """
       Drop the features/columns from the dataframe based on the threshold.
    
        Args:
            df (Pandas Dataframe)  : A pandas dataframe 
            threshold(int [0,1]) : 
        Return:
            df (Pandas Dataframe)  : A pandas dataframe with columns with percentage of 
                                     null/nan > threshold 
            
    """
    total_instances=len(df)
    print(threshold,total_instances)
    
    to_drop=[]
    to_retain=[]
    zero_count = get_zero_count(df)
    nan_count = get_nan_count(df)
    for key, value in zero_count.items():
        total_invalid = value + nan_count.get(key)
        if total_invalid/total_instances >= threshold:
            to_drop.append(key)
        else:
            to_retain.append(key)
    return set(to_drop), set(to_retain)
        

In [19]:
class DataAnalyzer():
    
    def __init__(self,filepath,df=None):
        self.src_file = filepath
        if not df:
            self.df = pd.read_csv(self.src_file)
        else:
            self.df = df
        self.df.columns = [x.lower() for x in self.df.columns]
        self.features = self.df.keys()
        self.len = len(self.df)
        self.total_features = len(self.df.keys())
        self.features_with_nan = self.df.columns[self.df.isna().any()].tolist()
        
    def load_stats(self):
        self.nans = (len(self.df) - self.df.count()).to_dict()
        self.zeros = (self.df == 0).sum(axis=0).to_dict() 
        self.negatives = (self.df.loc[:, self.df.dtypes != object] < 0).sum(axis=0).to_dict() 
        self.features_all_nan = [x for x,y in self.nans.items() if y == self.len ]
    
    def get_features(self, search_string):
        """
        Returns the list of features matching the search string.
        Args:
            search_string(str): search string.
        """
        return [x for x in self.features if search_string in x.lower()]
    
    def find_missing_count(self,feature):
        l = [feature,'year4']
        _df = self.df[l].loc[self.df[feature] != self.df[feature]]
        n = len(_df)
#         years = _df['year'].unique().tolist()
        return n, round(n * 100/self.len,4)
  
    def fill_missing(self, feature):
        if feature not in missing_feature_map.keys():
            print("Feature not found")
            return
          
        op = missing_feature_map[feature]['op']
        features = missing_feature_map[feature]['src_features']
    
        if op == ADD:
            self.df['temp']  = self.df[features].sum(axis=1)  
        elif op == SUBSTRACT:
            self.df['temp'] = self.df[features[0]] - self.df[features[1]]
            if len(features) == 3:
                self.df['temp'] = self.df['temp'] - self.df[features[2]]
        
        if feature in self.df.keys():
            del self.df[feature]  
        self.df.columns = self.df.columns.str.replace('temp',feature)  
    

## Observations ##

      

**1. Feature has all nans ==> Drop it **

**2. County level data has valid values except for year 2016.**

**3. 6059 records have Name as "NOT AVAILABLE" **
     

  


## TODO ## 